In [ ]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import pymongo

In [ ]:
##### FInd News Articles ###
url = "https://redplanetscience.com/"
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
soup_list = soup.find_all("div", class_="list_text")

In [ ]:
headline = []
description = []

In [ ]:
for i in soup_list:
    headlines = i.find("div", "content_title").text
    descriptions = i.find("div", "article_teaser_body").text
    headline.append(headlines)
    description.append(descriptions)

In [ ]:
##### FInd Featured Photo ###
url = "https://spaceimages-mars.com/"

In [ ]:
browser.visit(url)
time.sleep(0.025)

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
soup1 = soup.find("div", class_="floating_text_area")
photo_link = soup1.select_one("a",target_="_blank", class_="showimg fancybox-thumbs")["href"]

In [ ]:
######## Find Mars Data #############
url = "https://galaxyfacts-mars.com/"
tables = pd.read_html(url)
tables

df = tables[0]
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df = df.set_index("Mars - Earth Comparison")
df

In [ ]:
######## Find  Hemisphere photos#############
url = "https://marshemispheres.com/"
browser.visit(url)
time.sleep(.025)
html = browser.html
soup = bs(html, "html.parser")
soupp = soup.find('div', class_='collapsible results')
items = soupp.find_all("div", class_="item")

In [ ]:
scrape_link = []
picture_description = []
#image links
for i in items:
    desc_container = i.find("a", class_="itemLink product-item")['href']
    scrape_link.append(desc_container)
    picture_description.append(i.find("h3").text)

In [ ]:
image_link = []
for i in scrape_link:
    scrapable_url = f'{url}{i}'
    browser.visit(scrapable_url)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find('div', class_='downloads')
    image_url = downloads.find("a")["href"]
    image_link.append(f'{url}{image_url}')
browser.quit()    

In [ ]:
headline
description
photo_link
df
image_link

In [ ]:
mars_dictionary = {
    "news": {
        "headline": headline,
        "description": description
    },
    "featured_photo" : photo_link,
    "data": df.to_dict(),
    "hemispheres": {
        "name": picture_description,
        "image_link":image_link
    }
}

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.Mars_Data
Scrape_Results = db.Scrape_Results
Scrape_Results.insert_one(mars_dictionary)